In [1]:
!pip install -U "transformers>=4.46" "datasets>=2.19" peft safetensors

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.2.0
    Uninstalling datasets-4.2.0:
      Successfully uninstalled datasets-4.2.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [2]:
#B) セットアップ
import os, random, time, csv
from collections import Counter
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
#🟢 「パッケージ ＞ モジュール」
#🟢 それをまとめて pip install で入れる“製品一式”がライブラリ

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
torch.set_float32_matmul_precision("high")
print("Device:", DEVICE)

Device: mps


In [3]:
#C) データセット選択 & 正規化（実務寄り）
# "alpaca" | "dolly" | "squad" から選択
DATASET = "alpaca"
MAX_ROWS = 1200
HOLDOUT  = 200

def load_and_map(name):
    rows=[]
    if name=="dolly":
        ds = load_dataset("databricks/databricks-dolly-15k", split="train")
        for r in ds:
            inst = (r.get("instruction") or "").strip()
            ctx  = (r.get("context") or r.get("input") or "").strip()
            out  = (r.get("response") or r.get("output") or "").strip()
            if not inst or not out: continue
            if ctx: inst = inst + "\n" + ctx
            rows.append({"instruction":inst, "output":out})
    elif name=="alpaca":
        ds = load_dataset("yahma/alpaca-cleaned", split="train")
        for r in ds:
            inst = r["instruction"].strip()
            inp  = (r.get("input") or "").strip()
            out  = r["output"].strip()
            if inp: inst = inst + "\n" + inp
            rows.append({"instruction":inst, "output":out})
    elif name=="squad":
        ds = load_dataset("squad_v2", split="train")
        for r in ds:
            ctx = r["context"].strip(); q = r["question"].strip()
            answers = r["answers"]["text"]
            if answers:
                inst = f"Answer based on the context.\nContext: {ctx}\nQuestion: {q}"
                out  = answers[0].strip()
                rows.append({"instruction":inst, "output":out})
    else:
        raise ValueError("Unknown dataset")
    rows = rows[:MAX_ROWS]
    random.shuffle(rows)
    return rows

rows = load_and_map(DATASET)
train = rows[:-HOLDOUT]
hold  = rows[-HOLDOUT:]
print("train/hold:", len(train), len(hold))


train/hold: 1000 200


In [4]:
#D) トークナイズ（-100 マスク）
MODEL_NAME = "distilgpt2"   # MPSでも軽量で安定
BLOCK = 256                  # 重ければ 192 へ
BATCH = 8                    # 重ければ 4 へ

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

def apply_template(instruction, answer=""):
    prompt = f"### Instruction:\n{instruction}\n### Response:\n"
    return prompt, f"{answer}{tok.eos_token}"

def encode_row_fixed(row):
    prompt, answer = apply_template(row["instruction"], row["output"])
    prompt_ids = tok(prompt, add_special_tokens=False).input_ids
    enc = tok(prompt + answer, add_special_tokens=False,
              max_length=BLOCK, padding="max_length", truncation=True)
    labels = enc["input_ids"].copy()
    pl = min(len(prompt_ids), BLOCK)
    for i in range(pl): labels[i] = -100
    return {
        "input_ids": torch.tensor(enc["input_ids"], dtype=torch.long),
        "attention_mask": torch.tensor(enc["attention_mask"], dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long),
    }

train_rows = [encode_row_fixed(r) for r in train]
print("1 sample lens:", len(train_rows[0]["input_ids"]))

1 sample lens: 256


In [6]:
#E) DataLoader
class ListDS(Dataset):
    def __init__(self, items): self.items=items
    def __len__(self): return len(self.items)
    def __getitem__(self,i): return self.items[i]
#[
#  {"input_ids": tensor([...]), "attention_mask": tensor([...]), "labels": tensor([...])},
#  {"input_ids": tensor([...]), "attention_mask": tensor([...]), "labels": tensor([...])},
#  ...
#]
def collate(batch):
    keys = batch[0].keys()
    out={}
    for k in keys:
        out[k]=torch.stack([b[k] for b in batch], dim=0)
    return out
#{
#  "input_ids": tensor([[...], [...], ...]),         # shape: (batch_size, seq_len)
#  "attention_mask": tensor([[...], [...], ...]),
#  "labels": tensor([[...], [...], ...])
#}
loader = DataLoader(ListDS(train_rows), batch_size=BATCH, shuffle=True, collate_fn=collate)
batch = next(iter(loader))
print({ k: v.shape for k,v in batch.items() })

{'input_ids': torch.Size([8, 256]), 'attention_mask': torch.Size([8, 256]), 'labels': torch.Size([8, 256])}


In [7]:
#F) ユーティリティ（loss/学習/生成/評価・chrF代替）
from collections import Counter

def avg_loss(model, loader, n_batches=6):
    model.eval(); it=iter(loader); losses=[]
    with torch.no_grad():
        for _ in range(n_batches):
            try: b=next(it)
            except StopIteration: break
            b={k:v.to(DEVICE) for k,v in b.items()}
            losses.append(model(**b).loss.item())
    return float(np.mean(losses)) if losses else float("nan")

def train_steps(model, loader, lr=3e-4, steps=150):
    opt = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=lr)
    model.train(); it=iter(loader)
    tok_count=0; t0=time.time()
    for s in range(steps):
        try: b=next(it)
        except StopIteration: it=iter(loader); b=next(it)
        b={k:v.to(DEVICE) for k,v in b.items()}
        opt.zero_grad(); loss=model(**b).loss; loss.backward(); opt.step()
        tok_count += int(b["attention_mask"].sum().item())
        if (s+1)%50==0: print(f"step {s+1}/{steps} loss={loss.item():.4f}")
        if DEVICE=="mps" and (s+1)%25==0:
            try: torch.mps.empty_cache()
            except: pass
    tps = tok_count / max(time.time()-t0, 1e-6)
    return model, int(tps)

def generate_answers(model, prompts, max_new_tokens=40):
    model.eval(); outs=[]
    for p in prompts:
        enc = tok(p, return_tensors="pt")
        enc = {k:v.to(DEVICE) for k,v in enc.items()}
        with torch.no_grad():
            gen = model.generate(**enc, max_new_tokens=max_new_tokens, do_sample=False,
                                 eos_token_id=tok.eos_token_id, pad_token_id=tok.pad_token_id)
        text = tok.decode(gen[0][enc["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        outs.append(text)
    return outs

# evaluate無しの安定スコア（文字単位F1）
def char_f1(pred, ref):
    p=list(pred); r=list(ref)
    if len(p)==0 or len(r)==0: return 0.0
    cp, cr = Counter(p), Counter(r)
    overlap = sum((cp & cr).values())
    prec = overlap/len(p); rec = overlap/len(r)
    return 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)

def safe_chrf(preds, refs):
    preds=[("" if p is None else str(p).strip()) for p in preds]
    refs =[("" if r is None else str(r).strip()) for r in refs]
    if not preds or not refs: return 0.0
    scores=[char_f1(p,r) for p,r in zip(preds,refs)]
    return float(np.mean(scores))

In [8]:
#G) ホールドアウト作成
hold_prompts = [apply_template(r["instruction"])[0] for r in hold]
hold_refs    = [r["output"] for r in hold]
print("hold size:", len(hold_prompts))

hold size: 200


In [9]:
#H) 実行：A=LoRA / B=Head-only
def fresh_base():
    return AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

# A) LoRA
print("\n==> A: LoRA")
base_A = fresh_base()
lcfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
                  target_modules=["c_attn","c_proj"], task_type="CAUSAL_LM")
modelA = get_peft_model(base_A, lcfg)
b0A = avg_loss(modelA, loader)
modelA, tpsA = train_steps(modelA, loader, lr=3e-4, steps=150)
b1A = avg_loss(modelA, loader)
predA = generate_answers(modelA, hold_prompts)
cA   = safe_chrf(predA, hold_refs)

# B) Head-only（LMヘッドのみ学習）
print("\n==> B: Head-only")
base_B = fresh_base()
for p in base_B.parameters(): p.requires_grad=False
base_B.lm_head.weight.requires_grad=True
b0B = avg_loss(base_B, loader)
base_B, tpsB = train_steps(base_B, loader, lr=5e-4, steps=150)
b1B = avg_loss(base_B, loader)
predB = generate_answers(base_B, hold_prompts)
cB   = safe_chrf(predB, hold_refs)

print("\nSummary:",
      {"LoRA":   {"loss": (round(b0A,4), round(b1A,4)), "chrF": round(cA,4), "tok/s": tpsA},
       "Head":   {"loss": (round(b0B,4), round(b1B,4)), "chrF": round(cB,4), "tok/s": tpsB}})


==> A: LoRA


/Users/mh/opt/anaconda3/envs/ai-train-2025/lib/python3.11/site-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


step 50/150 loss=0.4391
step 100/150 loss=1.2159
step 150/150 loss=0.7185

==> B: Head-only
step 50/150 loss=1.3129
step 100/150 loss=2.2216
step 150/150 loss=1.4937

Summary: {'LoRA': {'loss': (9.5016, 1.1357), 'chrF': 0.3885, 'tok/s': 4102}, 'Head': {'loss': (7.6471, 1.0598), 'chrF': 0.0026, 'tok/s': 3969}}


In [10]:
#I) 保存（TSV / notes）
os.makedirs("results", exist_ok=True)
out_tsv = "results/hf_lora_mps.tsv"
with open(out_tsv,"w",newline="") as f:
    w=csv.writer(f, delimiter="\t")
    w.writerow(["dataset","model","variant","loss_before","loss_after","chrF","tokens_per_sec"])
    w.writerow([DATASET, MODEL_NAME, "LoRA",  round(b0A,4), round(b1A,4), round(cA,4), tpsA])
    w.writerow([DATASET, MODEL_NAME, "Head",  round(b0B,4), round(b1B,4), round(cB,4), tpsB])

notes = "results/notes.txt"
with open(notes,"a") as f:
    ts=time.strftime("%Y-%m-%d %H:%M:%S")
    f.write(f"[{ts}] DATASET={DATASET} MODEL={MODEL_NAME} BLOCK={BLOCK} BATCH={BATCH} DEVICE={DEVICE}\n")
    f.write(f" LoRA : loss {b0A:.4f}->{b1A:.4f} | chrF={cA:.4f} | tok/s={tpsA}\n")
    f.write(f" Head : loss {b0B:.4f}->{b1B:.4f} | chrF={cB:.4f} | tok/s={tpsB}\n")
    f.write("\n--- Samples (first 2) ---\n")
    for i in range(min(2, len(hold_prompts))):
        f.write(f"PROMPT:\n{hold_prompts[i]}\nREF : {hold_refs[i]}\n\n")
print("saved:", out_tsv, ",", notes)

saved: results/hf_lora_mps.tsv , results/notes.txt
